In [ ]:
# default_exp secrets

# Secrets Revealed 
> Summary

We take a peek under the hood and explain how Git performs its miracles. I will skimp over details. For in-depth descriptions refer to http://schacon.github.com/git/user-manual.html[the user manual].

### Invisibility 

How can Git be so unobtrusive? Aside from occasional commits and merges, you can work as if you were unaware that version control exists. That is, until you need it, and that's when you're glad Git was watching over you the whole time.

Other version control systems force you to constantly struggle with red tape and bureaucracy. Permissions of files may be read-only unless you explicitly tell a central server which files you intend to edit. The most basic commands may slow to a crawl as the number of users increases. Work grinds to a halt when the network or the central server goes down.

In contrast, Git simply keeps the history of your project in the `.git` directory in your working directory. This is your own copy of the history, so you can stay offline until you want to communicate with others. You have total control over the fate of your files because Git can easily recreate a saved state from `.git` at any time.

### Integrity 

Most people associate cryptography with keeping information secret, but another equally important goal is keeping information safe. Proper use of cryptographic hash functions can prevent accidental or malicious data corruption.

A SHA1 hash can be thought of as a unique 160-bit ID number for every string of bytes you'll encounter in your life. Actually more than that: every string of bytes that any human will ever use over many lifetimes.

As a SHA1 hash is itself a string of bytes, we can hash strings of bytes containing other hashes. This simple observation is surprisingly useful: look up 'hash chains'. We'll later see how Git uses it to efficiently guarantee data integrity.

Briefly, Git keeps your data in the `.git/objects` subdirectory, where instead of normal filenames, you'll find only IDs. By using IDs as filenames, as well as a few lockfiles and timestamping tricks, Git transforms any humble filesystem into an efficient and robust database.

### Intelligence 

How does Git know you renamed a file, even though you never mentioned the fact explicitly? Sure, you may have run *git mv*, but that is exactly the same as a *git rm* followed by a *git add*.

Git heuristically ferrets out renames and copies between successive versions. In fact, it can detect chunks of code being moved or copied around between files! Though it cannot cover all cases, it does a decent job, and this feature is always improving. If it fails to work for you, try options enabling more expensive copy detection, and consider upgrading.

### Indexing 

For every tracked file, Git records information such as its size, creation time and last modification time in a file known as the 'index'. To determine whether a file has changed, Git compares its current stats with those cached in the index. If they match, then Git can skip reading the file again.

Since stat calls are considerably faster than file reads, if you only edit a
few files, Git can update its state in almost no time.

We stated earlier that the index is a staging area. Why is a bunch of file
stats a staging area? Because the add command puts files into Git's database
and updates these stats, while the commit command, without options, creates a
commit based only on these stats and the files already in the database.

### Git's Origins 

This http://lkml.org/lkml/2005/4/6/121[Linux Kernel Mailing List post] describes the chain of events that led to Git. The entire thread is a fascinating archaeological site for Git historians.

### The Object Database 

Every version of your data is kept in the 'object database', which lives in the
subdirectory `.git/objects`; the other residents of `.git/` hold lesser data:
the index, branch names, tags, configuration options, logs, the current
location of the head commit, and so on. The object database is elementary yet
elegant, and the source of Git's power.

Each file within `.git/objects` is an 'object'. There are 3 kinds of objects
that concern us: 'blob' objects, 'tree' objects, and 'commit' objects.

### Blobs 

First, a magic trick. Pick a filename, any filename. In an empty directory:

In [ ]:
 ! echo sweet > YOUR_FILENAME
 ! git init
 ! git add .
 ! find .git/objects -type f


Reinitialized existing Git repository in /home/jovyan/work/git_magic/.git/
.git/objects/cc/d42c04c6d2eaae3fac6cca5caa0d49cd652b00
.git/objects/aa/823728ea7d592acc69b36875a482cdf3fd5c8d
.git/objects/cd/4bfa9f665f5a0861819a44f185f756ab204e0c
.git/objects/3f/866f780d4825a4e988d1858b756c9a9fa1a1d5
.git/objects/fb/68294b9c4c320e0fe123468270685e0739106e
.git/objects/ea/1a172cec8bb860e00fd6ecef20b9fc74c22425
.git/objects/38/0d73c9a0ad8af88ce229ddc6650309279b7fd0
.git/objects/22/64667f36f13659115f78653aac8c794fb2ef9c
.git/objects/83/f1f205dd8f99cb4d6eb0c96a4ec08c8c79c0bd
.git/objects/13/665904bc82a16cbf25145d2922d27c48819140
.git/objects/b7/e933d6338cf5af1058e623cecc9d1632ff1e98
.git/objects/da/087b57f5f5e0092f09c5dd766be607a3a6f590
.git/objects/93/8417075f4fe848cdddf9a851c37e6928ddafe7
.git/objects/ce/1439a5247efa53eee60aa8b19fb23b7595451a
.git/objects/ce/4223551fc20c995b0a29c4dc9c17c8a836dfb6
.git/objects/2f/4c48275b4ccc743b41239a4836a0f30e987359
.git/objects/9d/6ba467a42f3b11bd7134d51fb1e44


You'll see +.git/objects/aa/823728ea7d592acc69b36875a482cdf3fd5c8d+.

How do I know this without knowing the filename? It's because the
SHA1 hash of:

 "blob" SP "6" NUL "sweet" LF

is aa823728ea7d592acc69b36875a482cdf3fd5c8d,
where SP is a space, NUL is a zero byte and LF is a linefeed. You can verify
this by typing:



In [ ]:
  ! printf "blob 6\000sweet\n" | sha1sum


aa823728ea7d592acc69b36875a482cdf3fd5c8d  -



Git is 'content-addressable': files are not stored according to their filename,
but rather by the hash of the data they contain, in a file we call a 'blob
object'. We can think of the hash as a unique ID for a file's contents, so
in a sense we are addressing files by their content. The initial `blob 6` is
merely a header consisting of the object type and its length in bytes; it
simplifies internal bookkeeping.

Thus I could easily predict what you would see. The file's name is irrelevant:
only the data inside is used to construct the blob object.

You may be wondering what happens to identical files. Try adding copies of
your file, with any filenames whatsoever. The contents of +.git/objects+ stay
the same no matter how many you add. Git only stores the data once.

By the way, the files within +.git/objects+ are compressed with zlib so you
should not stare at them directly. Filter them through
http://www.zlib.net/zpipe.c[zpipe -d], or type:



In [ ]:
 ! git cat-file -p aa823728ea7d592acc69b36875a482cdf3fd5c8d


sweet



which pretty-prints the given object.

### Trees 

But where are the filenames? They must be stored somewhere at some stage.
Git gets around to the filenames during a commit:



In [ ]:
#  ! git commit  # Type some message.
#  ! find .git/objects -type f



You should now see 3 objects. This time I cannot tell you what the 2 new files are, as it partly depends on the filename you picked. We'll proceed assuming you chose ``rose''. If you didn't, you can rewrite history to make it look like you did:



In [ ]:
#  ! git filter-branch --tree-filter 'mv YOUR_FILENAME rose'
#  ! find .git/objects -type f



Now you should see the file
+.git/objects/05/b217bb859794d08bb9e4f7f04cbda4b207fbe9+, because this is the
SHA1 hash of its contents:

 "tree" SP "32" NUL "100644 rose" NUL 0xaa823728ea7d592acc69b36875a482cdf3fd5c8d

Check this file does indeed contain the above by typing:



In [ ]:
#  ! echo 05b217bb859794d08bb9e4f7f04cbda4b207fbe9 | git cat-file --batch



With zpipe, it's easy to verify the hash:



In [ ]:
 ! zpipe -d < .git/objects/05/b217bb859794d08bb9e4f7f04cbda4b207fbe9 | sha1sum


/bin/bash: .git/objects/05/b217bb859794d08bb9e4f7f04cbda4b207fbe9: No such file or directory
da39a3ee5e6b4b0d3255bfef95601890afd80709  -



Hash verification is trickier via cat-file because its output contains more
than the raw uncompressed object file.

This file is a 'tree' object: a list of tuples consisting of a file
type, a filename, and a hash. In our example, the file type is 100644, which
means `rose` is a normal file, and the hash is the blob object that contains
the contents of `rose'. Other possible file types are executables, symlinks or
directories. In the last case, the hash points to a tree object.

If you ran filter-branch, you'll have old objects you no longer need. Although
they will be jettisoned automatically once the grace period expires, we'll
delete them now to make our toy example easier to follow:



In [ ]:
#  ! rm -r .git/refs/original
#  ! git reflog expire --expire=now --all
#  ! git prune



For real projects you should typically avoid commands like this, as you are
destroying backups. If you want a clean repository, it is usually best to make
a fresh clone. Also, take care when directly manipulating +.git+: what if a Git
command is running at the same time, or a sudden power outage occurs?
In general, refs should be deleted with *git update-ref -d*,
though usually it's safe to remove +refs/original+ by hand.

### Commits 

We've explained 2 of the 3 objects. The third is a 'commit' object. Its
contents depend on the commit message as well as the date and time it was
created. To match what we have here, we'll have to tweak it a little:



In [ ]:
#  ! git commit --amend -m Shakespeare  # Change the commit message.
#  ! git filter-branch --env-filter 'export


     GIT_AUTHOR_DATE="Fri 13 Feb 2009 15:31:30 -0800"
     GIT_AUTHOR_NAME="Alice"
     GIT_AUTHOR_EMAIL="alice@example.com"
     GIT_COMMITTER_DATE="Fri, 13 Feb 2009 15:31:30 -0800"
     GIT_COMMITTER_NAME="Bob"
     GIT_COMMITTER_EMAIL="bob@example.com"'  # Rig timestamps and authors.


In [ ]:
#  ! find .git/objects -type f



You should now see
+.git/objects/49/993fe130c4b3bf24857a15d7969c396b7bc187+
which is the SHA1 hash of its contents:

 "commit 158" NUL
 "tree 05b217bb859794d08bb9e4f7f04cbda4b207fbe9" LF
 "author Alice <alice@example.com> 1234567890 -0800" LF
 "committer Bob <bob@example.com> 1234567890 -0800" LF
 LF
 "Shakespeare" LF

As before, you can run zpipe or cat-file to see for yourself.

This is the first commit, so there are no parent commits, but later commits
will always contain at least one line identifying a parent commit.

### Indistinguishable From Magic 

Git's secrets seem too simple. It looks like you could mix together a few shell scripts and add a dash of C code to cook it up in a matter of hours: a melange of basic filesystem operations and SHA1 hashing, garnished with lock files and fsyncs for robustness. In fact, this accurately describes the earliest versions of Git. Nonetheless, apart from ingenious packing tricks to save space, and ingenious indexing tricks to save time, we now know how Git deftly changes a filesystem into a database perfect for version control.

For example, if any file within the object database is corrupted by a disk
error, then its hash will no longer match, alerting us to the problem. By
hashing hashes of other objects, we maintain integrity at all levels. Commits
are atomic, that is, a commit can never only partially record changes: we can
only compute the hash of a commit and store it in the database after we already
have stored all relevant trees, blobs and parent commits. The object
database is immune to unexpected interruptions such as power outages.

We defeat even the most devious adversaries. Suppose somebody attempts to
stealthily modify the contents of a file in an ancient version of a project. To
keep the object database looking healthy, they must also change the hash of the
corresponding blob object since it's now a different string of bytes. This
means they'll have to change the hash of any tree object referencing the file,
and in turn change the hash of all commit objects involving such a tree, in
addition to the hashes of all the descendants of these commits. This implies the
hash of the official head differs to that of the bad repository. By
following the trail of mismatching hashes we can pinpoint the mutilated file,
as well as the commit where it was first corrupted.

In short, so long as the 20 bytes representing the last commit are safe,
it's impossible to tamper with a Git repository.

What about Git's famous features? Branching? Merging? Tags?
Mere details. The current head is kept in the file +.git/HEAD+,
which contains a hash of a commit object. The hash gets updated during a commit
as well as many other commands. Branches are almost the same: they are files in
+.git/refs/heads+. Tags too: they live in +.git/refs/tags+ but they
are updated by a different set of commands.
